In [ ]:
import sys
import AE_models

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, KFold
import pandas as pd
import numpy as np
import keras 
from keras import layers
import scipy.io as sio
import pandas as pd 

import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
def value_range_analysis(X):
    if sum(sum(((np.round(X,4) <= 1)) & (X >= 0))) == X.shape[0] * X.shape[1]:
        print("Range value: [0,1]")
    else:
        print("ERROR!!")

## Plot results

In [ ]:
def plotLayers(plot_first_layer, plot_second_layer, dicc, title, carpeta):
    plot_first_layer = plot_first_layer
    plot_second_layer = plot_second_layer

    scores = dicc[0]
    scoresTrain = dicc[1]

    fig = plt.figure(figsize=[20, 20])
    ax = fig.add_subplot(1,2,1, projection='3d')
    XX, YY = np.meshgrid(plot_first_layer, plot_second_layer)

    cluster = ['r','b','g','k','m', 'c','y', 'r', 'b']
    marker = ['.', 'o', '^', 'p', '*', 'v', 's', '.', 'o'] 

    for xp, yp, sco, c, m in zip(XX, YY, scores, cluster, marker):
        surf = ax.scatter([xp], [yp], [sco], cmap=cm.coolwarm,
                               linewidth=3, antialiased=False, color = c, marker = '.')

    ax.set_xlabel('plot_first_layer'); ax.set_ylabel('plot_second_layer'); ax.set_zlabel('val loss')

    ax.set_zlim([np.min(scores), np.max(scores)+0.1]);
    ax.set_xticks(plot_first_layer); 
    ax.set_yticks(plot_second_layer);
    ax.view_init(elev=10, azim=12);
    ax.set_title('Val loss - ' + str(title))

    ax = fig.add_subplot(1,2,2, projection='3d')
    XX, YY = np.meshgrid(plot_first_layer, plot_second_layer)

    
    for xp, yp, sco, c, m in zip(XX, YY, scoresTrain, cluster, marker):
        surf = ax.scatter([xp], [yp], [sco], cmap=cm.coolwarm,
                               linewidth=3, antialiased=False, color = c, marker = 'X')

    ax.set_xlabel('plot_first_layer'); ax.set_ylabel('plot_second_layer'); ax.set_zlabel('loss')

    ax.set_zlim([np.min(scores), np.max(scores)+0.1]);
    ax.set_xticks(plot_first_layer); 
    ax.set_yticks(plot_second_layer);
    ax.view_init(elev=10, azim=12);
    ax.set_title('Loss - ' + str(title))

In [ ]:
def plotLayers_vf(h_layers, dicc, carpeta):
    plot_first_layer = []
    plot_second_layer = []
    for i in range(len(h_layers)):
        plot_first_layer.append(h_layers[i][1])
        plot_second_layer.append(h_layers[i][2])

    keys = list(dicc.keys())
    for i in range(len(keys)):
        plotLayers(plot_first_layer, plot_second_layer, dicc[keys[i]], keys[i], carpeta)

## Final Version

In [ ]:
folders = ["S1", "S2", "S3", "S4", "S5"]
normalizar = True
plotAndSaveImg = False

for index_data in range(len(folders)):
    print("================================================================>" + folders[index_data] + "<=======================================================")
    X_train_original, X_test_original, y_train_original, y_test_original = AE_models.loadData(index_data, normalizar, folders)

    autoencodertype = {'DAE': False, 'AE': True}


    epochs = 5000
    initial_learning_rate = [1e-4, 1e-3, 1e-2, 1e-1]
    batch_size = [32]
    dropout=[0, .05, .1]

    h_layers = [[X_train_original.shape[0], 600],
                [X_train_original.shape[0], 575],
                [X_train_original.shape[0], 550],
                [X_train_original.shape[0], 525],
                [X_train_original.shape[0], 500],
                [X_train_original.shape[0], 470],
                [X_train_original.shape[0], 450],
                [X_train_original.shape[0], 425],
                [X_train_original.shape[0], 400],
                [X_train_original.shape[0], 375],
                [X_train_original.shape[0],  350],
                [X_train_original.shape[0],  325]]
    
    hyperparameters = {'epochs': epochs,
                       'initial_learning_rate':initial_learning_rate,
                       'kfold':5,
                       "mindelta": 0.0001}

    y_pre_train = y_train_original
    y_test = y_test_original
    X_pre_train = X_train_original
    X_test = X_test_original

    semillas = [9,18,35, 52, 75]

    kf = KFold(n_splits=hyperparameters["kfold"], shuffle=True, random_state=semillas[index_data])
    kf.get_n_splits(X_pre_train)

    bestHyperparameters = {'h_layers': -1, 'batch_size': -1, 'initial_learning_rate': -1 }
    dicc = {}
    bestMetricDev = np.inf
    for d in range(len(dropout)):
        for init in range(len(initial_learning_rate)):
            for bs in range(len(batch_size)):
                plot_val_loss_mean = []
                plot_loss_mean = []
                for l in range(len(h_layers)):

                    hyperparameters_aux = {'epochs': hyperparameters['epochs'],
                                       'initial_learning_rate': hyperparameters['initial_learning_rate'][init],
                                       "mindelta": hyperparameters["mindelta"],
                                       'batch_size': batch_size[bs],
                                       'h_layers': h_layers[l],
                                       'dropout': dropout[d],
                                       'verbose': 0}

                    v_metric_dev = []
                    val_loss = []
                    loss = []
                    for train_index, val_index in kf.split(X_pre_train):
                        X_train, X_val = X_pre_train[train_index], X_pre_train[val_index]
                        # Reset keras
                        AE_models.reset_keras()
                        # Run the model
                        model, hist, early = AE_models.runNetwork(X_train, X_val,
                                                                  hyperparameters_aux,
                                                                  autoencodertype)


                        loss.append(np.min(hist.history["loss"]))
                        val_loss.append(np.min(hist.history["val_loss"]))

                    plot_val_loss_mean.append(np.mean(val_loss))
                    plot_loss_mean.append(np.mean(loss))   
                    metric_dev = np.mean(val_loss)
                    if metric_dev < bestMetricDev:
                        print("\tChange the best ", bestMetricDev, " by metric dev: ", metric_dev)
                        bestMetricDev = metric_dev
                        bestHyperparameters['h_layers'] = l
                        bestHyperparameters['batch_size'] = bs
                        bestHyperparameters['initial_learning_rate'] = init
                        bestHyperparameters['dropout'] = d
                        print("%%%%%%%%%bestHyperparameters%%%%%%%%%")
                        print(bestHyperparameters)
                
                dicc["bs:" + str(batch_size[bs]) + "-lr:" + str(initial_learning_rate[init]) + "-drop:" + str(dropout[d])] = [plot_val_loss_mean, plot_loss_mean]


    if plotAndSaveImg:
        plotLayers_vf(h_layers, dicc, folders[index_data])

    print("Layers selected:", h_layers[bestHyperparameters["h_layers"]])
    print("batch_size selected:", batch_size[bestHyperparameters["batch_size"]])
    print("initial_learning_rate selected:", initial_learning_rate[bestHyperparameters["initial_learning_rate"]])
    print("dropout selected:", dropout[bestHyperparameters["dropout"]])

    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%================================================================> BEST MODEL SELECT WITH DIFFERENTS HYPERPARAMETERS <=======================================================%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

    hyperparameters = {'epochs': hyperparameters['epochs'],
                   'initial_learning_rate': initial_learning_rate[bestHyperparameters["initial_learning_rate"]],
                   "mindelta": hyperparameters["mindelta"],
                   'batch_size': batch_size[bestHyperparameters["batch_size"]],
                   'h_layers': h_layers[bestHyperparameters["h_layers"]],
                   'dropout': dropout[bestHyperparameters["dropout"]],
                   'verbose': 1}
    
    AE_models.reset_keras()
    X_train_aux, X_val_aux = train_test_split(X_pre_train,
                                              test_size=0.3,
                                              random_state=semillas[index_data])
    
    autoencoder, hist, early = AE_models.runNetwork(X_train_aux, X_val_aux,
                                                    hyperparameters,
                                                    autoencodertype)

    encoder = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)
    #Load the data
    X_train = sio.loadmat('../../data_reduced_by_tck/' + folders[index_data] + '/Ktrtr')
    X_test = sio.loadmat('../../data_reduced_by_tck/' + folders[index_data] + '/Ktrte')
    X_train = X_train['Ktrtr']
    X_test = X_test['Ktrte']
    X_test = X_test.T

    X_train, X_test = AE_models.normData_minmax(X_train, X_test)

    encoder = keras.Model(autoencoder.input, autoencoder.get_layer('Latent_layer').output)
    # Encode the data
    X_train_encode = encoder.predict(X_train)
    X_test_encode = encoder.predict(X_test)

    pd.DataFrame(X_train_encode).to_csv("../../1_Clasifications_models/data_reduced/AE/X_train_Norm_AE_" + folders[index_data] + ".csv", index=False)
    pd.DataFrame(X_test_encode).to_csv("../../1_Clasifications_models/data_reduced/AE/X_test_Norm_AE_" + folders[index_data] + ".csv", index=False)